In [1]:
from nltk.stem.porter import PorterStemmer
import string, csv, os, sys, nltk
import pandas as pd
import numpy as np
import re, pickle, contractions, nltk
import preprocessor as p
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pathlib import Path
from collections import Counter
import ast

In [26]:
def lemmatize_text(text):  
    porter_stemmer = PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    return [( porter_stemmer.stem(lemmatizer.lemmatize(w))  )  for w in w_tokenizer.tokenize((text))]


In [27]:
def clean_tweets(comments):
    comments['comment_clean'] = np.nan
    #add hashtags into a column

    for i,v in enumerate(comments['text']):
        #basic cleaning (urls, mentions, emojis, smileys, hashtags, reserved words(rt,fav
        clean = p.clean(v)

        #remove links
        clean = re.sub(r'http\S+', '', clean)

        #remove digits
        clean = re.sub('[0-9]+', '', clean)

        #make string lowercase
        clean = clean.lower()

        #expand contractions
        clean = contractions.fix(clean)

        #remove punctuations
        clean = re.sub(r'[^\w\s]', '', clean)

        #remove underscores
        clean = clean.replace("_"," ")

        #remove words: hagupit, ruby, rubyph
        remove_words = ['cong', 'congtv', 'team payaman']
        clean_list= clean.split()
        clean_list = [word for word in clean_list if word.lower() not in remove_words]
        clean = ' '.join(clean_list)

        #lemmatize and get ngrams
        clean = lemmatize_text(clean)

        #set tweet_clean column data type as object
        comments['comment_clean'] = comments['comment_clean'].astype('object')

        #insert preprocessed texts into column
        comments.at[i, 'comment_clean'] = clean

    #remove stopwords
    stop_words = set(stopwords.words('english'))
    comments['comment_clean'] = comments['comment_clean'].apply(lambda x: [item for item in x if item not in stop_words])

    #combine unigrams into string
    for i,v in enumerate(comments['comment_clean']):
        unigram_string = ' '.join(v)
        comments.at[i, 'comment_clean'] = unigram_string

    return comments

In [28]:
df = pd.read_csv('all.csv')
df = df.iloc[:300]
df['label'].value_counts()

others        157
compliment    136
suggestion      4
question        2
shoutout        1
Name: label, dtype: int64

In [29]:
vect = TfidfVectorizer(use_idf=True, min_df=5, max_df=0.85, ngram_range=(1,2))


X = clean_tweets( df )['comment_clean']
vect.fit(X) 
X = vect.fit_transform( X )
y = df['label']


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC()

In [31]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)


0.8166666666666667

In [32]:

precision_recall_fscore_support(y_test,y_pred, average=None)

c:\users\arjna\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.9047619 , 0.76923077, 0.        ]),
 array([0.7037037, 0.9375   , 0.       ]),
 array([0.79166667, 0.84507042, 0.        ]),
 array([27, 32,  1], dtype=int64))

In [33]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average=None)

c:\users\arjna\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([0.9047619 , 0.76923077, 0.        ])

In [34]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average='macro')

0.5470679012345679

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred) )


              precision    recall  f1-score   support

  compliment       0.90      0.70      0.79        27
      others       0.77      0.94      0.85        32
    question       0.00      0.00      0.00         1

    accuracy                           0.82        60
   macro avg       0.56      0.55      0.55        60
weighted avg       0.82      0.82      0.81        60



c:\users\arjna\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\arjna\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\arjna\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [12]:
def use_model(text , model):
    text = [text,]
    features = vect.transform(text)
    y_pred = model.predict(features)

    return y_pred

In [25]:
use_model( "sana manotice" ,clf)

array(['others'], dtype=object)